Intermodal nx_graph can be loaded from OSM [example](graph_generator.ipynb)

In [1]:
from shapely import Point
from objectnat import get_accessibility_isochrones,GraphType
from objectnat import get_intermodal_graph_from_osm
import networkx as nx
graph = get_intermodal_graph_from_osm(city_osm_id=7226665, city_crs=32643)
accessibility_isochrones, public_transport_routes, public_transport_stops = get_accessibility_isochrones(
    graph_type=[GraphType.PUBLIC_TRANSPORT, GraphType.WALK],
    points=Point(461525,6305709),
    weight_value=20,
    weight_type="time_min",
    city_graph=graph,
    city_crs=32643,
)

# Visualize data
m1 = accessibility_isochrones.explore()
# public_transport_routes.explore(m=m1) # if not None
# public_transport_stops.explore(m=m1)  # if not None
m1

2024-05-24 16:01:10.802 | INFO     | dongraphio.dongraphio:get_intermodal_graph_from_osm:55 - Creating intermodal graph from OSM...
2024-05-24 16:01:10.803 | WARNING  | dongraphio.base_models.grapher_logic:get_public_trasport_graph:236 - Files with routes or with stops was not found. The graph will be built based on data from OSM
2024-05-24 16:01:11.282 | WARNING  | dongraphio.base_models.grapher_logic:public_routes_to_edges:161 - It seems there are no subway routes in the city. This transport type will be skipped.
2024-05-24 16:01:11.505 | WARNING  | dongraphio.base_models.grapher_logic:public_routes_to_edges:161 - It seems there are no tram routes in the city. This transport type will be skipped.
2024-05-24 16:01:11.754 | WARNING  | dongraphio.base_models.grapher_logic:public_routes_to_edges:161 - It seems there are no trolleybus routes in the city. This transport type will be skipped.
2024-05-24 16:01:12.097 | WARNING  | dongraphio.base_models.grapher_logic:public_routes_to_edges:16

2024-05-24 16:01:14.547 | DEBUG    | dongraphio.base_models.grapher_logic:get_osmnx_graph:94 - Extracting and preparing drive graph from OSM ...


2024-05-24 16:01:15.423 | DEBUG    | dongraphio.base_models.grapher_logic:get_intermodal_graph:45 - Preparing union of city_graphs...


Joining walk graph and car graph:   0%|          | 0/2417 [00:00<?, ?it/s]

2024-05-24 16:01:15.721 | INFO     | dongraphio.base_models.grapher_logic:get_intermodal_graph:65 - Intermodal graph done!

2024-05-24 16:01:15.722 | INFO     | dongraphio.dongraphio:get_accessibility_isochrones:118 - Creating accessibility isochrones based on provided graph and point...
2024-05-24 16:01:15.805 | DEBUG    | dongraphio.base_models.isochrones_logic:get_accessibility_isochrone:57 - Calculating distances from the specified point...
2024-05-24 16:01:15.873 | DEBUG    | dongraphio.base_models.isochrones_logic:get_accessibility_isochrone:75 - Building isochrones geometry...
2024-05-24 16:01:15.935 | INFO     | dongraphio.base_models.isochrones_logic:get_accessibility_isochrone:100 - No public transport node in graph
2024-05-24 16:01:15.936 | INFO     | dongraphio.dongraphio:get_accessibility_isochrones:127 - Accessibility isochrones done!

